# Description of what this code does needs to go here
*Question: Traveltimes or travelspeeds?*

In [1]:
#  Logger for writing outputs to .log file.
#  .info prints to standard output + .log file
#  .debug prints to only .log file

import logging
import time
import sys
logdatetime = time.strftime("%m%d")

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logger.handlers =[]

fh = logging.FileHandler(logdatetime+'.log', mode='w')
fh.setLevel(logging.DEBUG)

sh = logging.StreamHandler(stream=sys.stdout)
sh.setLevel(logging.INFO)

formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
fh.setFormatter(formatter)

logger.addHandler(fh)
logger.addHandler(sh)

In [2]:
import config

'''Select path name for files depending on whether you are working on your PC or cluster'''
# path_for_data is an optional variable that defaults to 'path'. This is included to facilate a seperate path
# address for large data files

path = './Data_Files/'         # Use this if working on PC
path_for_data = '../../'       # Use this if working on PC and if data files are in a different folder

#path = './scratch/'           # Use this if working on cluster
#path_for_data = './scratch/'  # Use this if working on cluster and if data files are in a different folder

filenames = config.generate_filenames(path, path_for_data)

logger.info('Filenames have been loaded')
[logger.debug(i) for i in filenames]

logger.info('Constants have been loaded')
[logger.debug(i) for i in dir(globals()['config'])];

Filenames have been loaded
Constants have been loaded


In [3]:
## Read Full-Link data and prep for sunning cSNMF.

import numpy as np
import pandas as pd
from read_data import read_full_link_json
import cSNMF      
%matplotlib inline

full_link_ids, D = read_full_link_json(filenames)  # Reads trips or travelspeeds depending on config.TRIPS value 
D = pd.DataFrame(D)

logger.info('Full_link data has been read')

if config.SEEDED == 1:
    seed_W = 0
    seed_H = 1
elif config.SEEDED == 0:
    seed_W = None
    seed_H = None
else:
    logger.critical('Seed value invalid. Needs to be 0 or 1. Check config.py!')
    quit()

Trips
Full_link data has been read


In [4]:
## For running the algorithm on a range of beta values:

def run_factorize(beta):
    log_string= 'log'+str(beta)
    log = logger.getChild(str(beta))
    log.addHandler(logging.FileHandler('log'+str(beta)+'.log', mode='w'))
    logger.info('beta = %s' %beta)
    return cSNMF.factorize(D, filenames, rank = config.RANK, beta = beta, seed_W = seed_W,\
                                seed_H = seed_H, log=log)

In [5]:
from joblib import Parallel, delayed

#results = Parallel(n_jobs=4)(delayed(run_factorize)(b) for b in np.arange(0.00000,0.00001,0.00001))

W, H, results = run_factorize(0)

beta = 0
Rank= 50, Beta= 0, Threshold= 0.2
Initializing W and H...
W, H chosen
Iteration= 0, Error= 49.3844304642
Iteration= 1, Error= 47.6330293392
Iteration= 2, Error= 47.5822163035
Iteration= 3, Error= 47.5373016956
Iteration= 4, Error= 47.4914929601
Iteration= 5, Error= 47.4424733367
Iteration= 6, Error= 47.3878744103
Iteration= 7, Error= 47.3247646808
Iteration= 8, Error= 47.2493747006
Iteration= 9, Error= 47.1568302522
Iteration= 10, Error= 47.0408998379
Iteration= 11, Error= 46.8938206999
Iteration= 12, Error= 46.7063579646
Iteration= 13, Error= 46.4683494571
Iteration= 14, Error= 46.1700611213
Iteration= 15, Error= 45.8045713873
Iteration= 16, Error= 45.3709034164
Iteration= 17, Error= 44.8767287856
Iteration= 18, Error= 44.3387040112
Iteration= 19, Error= 43.7788822395
Iteration= 20, Error= 43.2189362851
Iteration= 21, Error= 42.6752030746
Iteration= 22, Error= 42.1568950902
Iteration= 23, Error= 41.6671172114
Iteration= 24, Error= 41.2049682949
Iteration= 25, Error= 40.767437

Iteration= 223, Error= 26.9346167242
Iteration= 224, Error= 26.8860158671
Iteration= 225, Error= 26.9024263163
Iteration= 226, Error= 27.0219075731
Iteration= 227, Error= 27.3520762893
Iteration= 228, Error= 27.723920984
Iteration= 229, Error= 32.0697334206
Iteration= 230, Error= 28.1552872974
Iteration= 231, Error= 27.4808909623
Iteration= 232, Error= 27.1536959109
Iteration= 233, Error= 26.9798663906
Iteration= 234, Error= 26.8895477584
Iteration= 235, Error= 26.8385345359
Iteration= 236, Error= 26.8062954415
Iteration= 237, Error= 26.7849733823
Iteration= 238, Error= 26.772493214
Iteration= 239, Error= 26.7849089591
Iteration= 240, Error= 26.8570017652
Iteration= 241, Error= 27.0430599953
Iteration= 242, Error= 27.3931914184
Iteration= 243, Error= 28.3906952751
Iteration= 244, Error= 31.3531844277
Iteration= 245, Error= 34.4780865015
Iteration= 246, Error= 29.8481635643
Iteration= 247, Error= 36.6337211275
Iteration= 248, Error= 55.4172779618
Iteration= 249, Error= 38.7325011321
Ite

/home/vaibhav/Dropbox/Traffic_Temp/Phase1.py:193: RuntimeWarning: invalid value encountered in double_scalars
  diff_W = lin.norm(W_new - W)/lin.norm(W)*100


Iteration= 360, Error= 70.1290944596
Sparsity= 0.976971896122


In [6]:
%matplotlib notebook
import matplotlib.pyplot as plt
#pd.set_option('display.max_colwidth',1000)
r = results.values
plt.plot(r[:130,0:2])


<IPython.core.display.Javascript object>

In [7]:
if config.TRIPS == 1:
    W.to_csv(filenames['W_trips']);                     print('W_trips written')
    H.to_csv(filenames['H_trips']);                     print('H_trips written')
    cSNMF.axe_H(H).to_csv(filenames['H_trips_axed']);  print('H_trips_axed written')

elif config.TRIPS == 0:
    W.to_csv(filenames['W_speeds']);                     print('W_speeds written')
    H.to_csv(filenames['H_speeds']);                     print('H_speeds written')
    cSNMF.axe_H(H).to_csv(filenames['H_speeds_axed']);  print('H_speeds_axed written')

W_trips written
H_trips written
H_trips_axed written


In [8]:
np.sum(W)

0      1.736578e+00
1      1.609293e+00
2      6.771398e-14
3      1.737650e+00
4      4.002846e-19
5      2.135430e+00
6      1.870756e+00
7      5.350504e-10
8      2.692383e+00
9      9.536911e-01
10     3.064555e+00
11     1.651320e+00
12     1.840715e+00
13     1.492986e+00
14     1.777156e+00
15     1.252117e+00
16     9.977536e-23
17     1.639547e-12
18     7.740659e-29
19     1.466245e+00
20     1.508550e+00
21     4.950081e+00
22     2.264731e+00
23    4.060931e-191
24     2.468128e+00
25     1.119408e+00
26     1.439111e+00
27    4.076740e+185
28     1.625185e+00
29     1.570685e+00
30     1.237845e+00
31     6.519782e+27
32     1.994255e+00
33     1.412507e+00
34    2.304731e+114
35     1.417018e+00
36     1.552277e+00
37     1.369706e+00
38     1.995213e+16
39     1.269479e+00
40     4.890558e-32
41     1.469562e+00
42     1.457666e+00
43     2.442544e+00
44     2.368617e-13
45     1.331123e+00
46     1.211975e+00
47     1.552677e+00
48     9.942920e-01
49     2.976128e+00


In [9]:
results.plot(y='error')
results.plot(y='sparsity')
results.plot(y='H_min')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

KeyError: 'H_min'

In [ ]:
H_loaded = pd.read_csv(filenames['H_trips'], names=['orig_index']+full_link_ids, skiprows=1)
H_loaded
sig0_link_ids = H_loaded.loc[0,:][H_loaded.loc[0,:]>0].keys().tolist()[1:]
sig0_link_ids

In [ ]:
import geopandas as gpd
from shapely.geometry import LineString

df = gpd.pd.read_csv(filenames['links'])
df = df[df['link_id'].isin(sig0_link_ids)]
geometry = [LineString([(x1,y1),(x2,y2)]) for (x1,y1,x2,y2) in zip(df.startX, df.startY, df.endX, df.endY)]

crs = {'init': 'epsg:4326'}
geo_df = gpd.GeoDataFrame(df, crs=crs, geometry=geometry)
geo_df
geo_df.to_file('sig0')

In [ ]:
import matplotlib.pyplot as plt

betas = np.arange(0.00001,0.00010,0.00001)
sparsities = [0.858, 0.866, 0.875, 0.878, 0.881, 0.887, 0.891, 0.894, 0.897]
errors = [26.16, 27.91, 30.069, 32.395, 34.686, 36.83, 38.863, 40.776, 42.575]
plt.plot(betas, errors, 'o-')
#plt.plot(betas, sparsities, 'o-')
plt.show()

In [ ]:
w = np.loadtxt('./Debug/W.csv', delimiter=',')
w = pd.DataFrame(w)
sorted(w.sum())